In [67]:
%matplotlib ipympl

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import gsw
import matplotlib.dates as mdates
import utide

In [68]:
ds = xr.open_dataset('data/MBARI_StationM_ADCP_201711_201811.nc')

In [69]:
current_file = 'data/Rover_II_Current_Mag_Hourly_Avg_pad_2018.csv'
df_all = pd.read_csv(current_file,parse_dates=[1])
df_all['Date_time_R'] = pd.to_datetime(df_all['Date_time_R'])

In [70]:
df = df_all[(df_all['Date_time_R'] >= np.array(ds['time'])[0]) & 
  (df_all['Date_time_R'] <= np.array(ds['time'])[-1])]

In [72]:
height = np.atleast_1d(1.5) # add instrument height

Lsmaj = np.nan*np.ones([len(height),len(constit_list)])
Lsmin = np.nan*np.ones([len(height),len(constit_list)])
theta = np.nan*np.ones([len(height),len(constit_list)])
g = np.nan*np.ones([len(height),len(constit_list)])
SNR = np.nan*np.ones([len(height),len(constit_list)])
PE = np.nan*np.ones([len(height),len(constit_list)])

Lsmaj_ci = np.nan*np.ones([len(height),len(constit_list)])
Lsmin_ci = np.nan*np.ones([len(height),len(constit_list)])
theta_ci = np.nan*np.ones([len(height),len(constit_list)])
g_ci = np.nan*np.ones([len(height),len(constit_list)])

# tidal analysis
time = mdates.date2num(df['Date_time_R'])

zi = 0

coef = utide.solve(time, np.array(df['Easting (cm/sec)']/100), 
         np.array(df['Northing (cm/sec)']/100),
         lat=35+8.4585/60,
         constit=constit_list,
         method='ols')

ni = np.argsort(coef['aux']['frq'])

print(coef['name'][ni])

Lsmaj[zi,:] = coef['Lsmaj'][ni]
Lsmin[zi,:] = coef['Lsmin'][ni]
theta[zi,:] = coef['theta'][ni]
g[zi,:] = coef['g'][ni]
SNR[zi,:] = coef['diagn']['SNR'][ni]
PE[zi,:] = coef['diagn']['PE'][ni]
Lsmaj_ci[zi,:] = coef['Lsmaj_ci'][ni]
Lsmin_ci[zi,:] = coef['Lsmin_ci'][ni]
theta_ci[zi,:] = coef['theta_ci'][ni]
g_ci[zi,:] = coef['g_ci'][ni]

solve: matrix prep ... solution ... diagnostics ... done.
['SSA' 'MSM' 'MM' 'MSF' 'MF' 'ALP1' '2Q1' 'SIG1' 'Q1' 'RHO1' 'O1' 'TAU1'
 'BET1' 'NO1' 'CHI1' 'P1' 'K1' 'PHI1' 'THE1' 'J1' 'SO1' 'OO1' 'UPS1' 'OQ2'
 'EPS2' '2N2' 'MU2' 'N2' 'NU2' 'M2' 'MKS2' 'LDA2' 'L2' 'S2' 'K2' 'MSN2'
 'ETA2' 'MO3' 'M3' 'SO3' 'MK3' 'SK3' 'MN4' 'M4' 'SN4' 'MS4' 'MK4' 'S4'
 'SK4' '2MK5' '2SK5' '2MN6' 'M6' '2MS6' '2MK6' '2SM6' 'MSK6' '3MK7' 'M8']


In [73]:
Lsmaj

array([[0.00206869, 0.00138132, 0.00220596, 0.00089534, 0.00217285,
        0.00021602, 0.00030272, 0.00048997, 0.00084435, 0.00022987,
        0.00470419, 0.00028978, 0.00069315, 0.00031899, 0.00038734,
        0.00242699, 0.00718388, 0.00023531, 0.00022275, 0.00048868,
        0.00047548, 0.00051903, 0.0004395 , 0.00063136, 0.00055476,
        0.0005438 , 0.00134389, 0.00414431, 0.00073402, 0.01705127,
        0.00068768, 0.00103919, 0.00060522, 0.00912358, 0.00250142,
        0.00106236, 0.00066507, 0.00049611, 0.00028858, 0.00048182,
        0.00128035, 0.00032641, 0.00058106, 0.00184922, 0.00056176,
        0.00213007, 0.00023852, 0.00044518, 0.00028676, 0.00017886,
        0.00017611, 0.0004368 , 0.00026995, 0.00029853, 0.00021444,
        0.00029113, 0.00015114, 0.00011454, 0.0001154 ]])

In [74]:
dstide = xr.Dataset({'Lsmaj': (('height', 'constituent'), Lsmaj),
                     'Lsmin': (('height', 'constituent'), Lsmin),
                     'theta': (('height', 'constituent'), theta),
                     'g': (('height', 'constituent'), g),
                     'SNR': (('height', 'constituent'), SNR),
                     'PE': (('height', 'constituent'), PE),
                     'Lsmaj_ci': (('height', 'constituent'), Lsmaj_ci),
                     'Lsmin_ci': (('height', 'constituent'), Lsmin_ci),
                     'theta_ci': (('height', 'constituent'), theta_ci),
                     'g_ci': (('height', 'constituent'), g_ci),
                     'freq':(('constituent'),coef['aux']['frq'])},
                {'height': height, 'constituent': coef['name'][ni]})

In [75]:
dstide.attrs['reftime']= coef['aux']['reftime']

In [76]:
dstide.to_netcdf('utide_results_rover18.nc')

In [77]:
coef

{'name': array(['M2', 'S2', 'K1', 'O1', 'N2', 'K2', 'P1', 'SSA', 'MM', 'MF', 'MS4',
        'M4', 'MSM', 'MU2', 'MK3', 'MSN2', 'LDA2', 'MSF', 'NU2', 'Q1',
        'OQ2', 'ETA2', 'BET1', 'MKS2', 'EPS2', 'MN4', 'L2', 'SN4', '2N2',
        'OO1', 'J1', 'SO3', 'MO3', 'SIG1', 'SO1', 'S4', 'UPS1', '2MN6',
        'CHI1', 'NO1', 'SK4', 'SK3', '2SM6', 'TAU1', 'M3', '2MS6', '2Q1',
        'MK4', 'M6', 'ALP1', 'THE1', 'RHO1', 'PHI1', '2MK6', '2MK5',
        '2SK5', 'MSK6', 'M8', '3MK7'], dtype=object),
 'aux': {'frq': array([8.05114007e-02, 8.33333333e-02, 4.17807462e-02, 3.87306544e-02,
         7.89992487e-02, 8.35614924e-02, 4.15525871e-02, 2.28159110e-04,
         1.51215197e-03, 3.05009177e-03, 1.63844734e-01, 1.61022801e-01,
         1.30978069e-03, 7.76894680e-02, 1.22292147e-01, 8.48454853e-02,
         8.18211814e-02, 2.82193266e-03, 7.92016200e-02, 3.72185025e-02,
         7.59749448e-02, 8.50736444e-02, 4.00404351e-02, 8.07395598e-02,
         7.61773160e-02, 1.59510649e-01, 8.2023552